In [2]:
import pandas as pd
from functools import reduce

In [185]:
Movement = pd.read_csv("ArtistMovements.csv")
Artwork = pd.read_csv("Artwork.csv")
Special = pd.read_csv("ArtistSpecializations.csv")
Generated = pd.read_csv("Generated.csv")
Artist = pd.read_csv("Artist.csv")
Education = pd.read_csv("ArtistEducation.csv")
Places = pd.read_csv("Places")
OutputReal = pd.read_csv("output_real.csv")
OutputFake = pd.read_csv("output_fake.csv")

# Artwork

In [128]:
Artwork.shape

(10517, 10)

In [120]:
pd.set_option('max_colwidth', 50)
Artwork.head(3)

,id,name,url,image_url,artist,rating,summary,year,medium,location
0,0,Cafe Terrace on the Place du Forum,http://wikigallery.org/wiki/painting_133032/Vi...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,8.0,Café Terrace at Night is an 1888 oil painting ...,1888.0,0.0,Kröller-Müller Museum
1,1,Starry Night,http://wikigallery.org/wiki/painting_9698/Vinc...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,9.0,NaN,NaN,NaN,NaN
2,2,A Digger,http://wikigallery.org/wiki/painting_132953/Vi...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,7.0,NaN,NaN,NaN,NaN


In [121]:
Artwork["year"].isna().sum()/len(Artwork) #should be removed due to large number of NAs 

0.9751830369877341

In [111]:
Artwork["medium"].isna().sum()/len(Artwork) #should be removed due to large number of NAs 

0.9499857373775792

In [112]:
Artwork["location"].isna().sum()/len(Artwork) #should be removed due to large number of NAs 

0.9395264809356281

In [281]:
Artwork2 = Artwork.iloc[:, [0,1,3,4,5]]
Artwork2 = Artwork2.rename(columns = {'artist':'artist_id'})
Artwork2.sample(5)

,id,name,image_url,artist_id,rating
8999,9000,La retour du marché,https://kuleuven-datathon-2023.s3.eu-central-1...,512,9.0
2077,2077,The Virgin and Child Adored by St.Louis of France,https://kuleuven-datathon-2023.s3.eu-central-1...,85,NaN
6235,6235,Amnon and Tamar,https://kuleuven-datathon-2023.s3.eu-central-1...,355,NaN
3055,3055,"The Dream, 1927",https://kuleuven-datathon-2023.s3.eu-central-1...,139,9.0
3145,3145,Summer,https://kuleuven-datathon-2023.s3.eu-central-1...,146,9.0


In [282]:
Artwork2["AI_Generated"] = 0
Artwork2.head()

,id,name,image_url,artist_id,rating,AI_Generated
0,0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1...,0,8.0,0
1,1,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1...,0,9.0,0
2,2,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1...,0,7.0,0
3,3,A Group of Cottages,https://kuleuven-datathon-2023.s3.eu-central-1...,0,6.0,0
4,4,A Pair of Shoes,https://kuleuven-datathon-2023.s3.eu-central-1...,0,8.0,0


In [116]:
pd.set_option('max_colwidth', 200)
Artwork["image_url"].head(5)

0    https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Cafe+Terrace+on+the+Place+du+Forum.jpg
1                          https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Starry+Night.jpg
2                              https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Digger.jpg
3                   https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Group+of+Cottages.jpg
4                       https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Pair+of+Shoes.jpg
Name: image_url, dtype: object

# Generated

In [126]:
Generated.shape

(3447, 2)

In [154]:
Generated.head()

,source_artwork,url,AI_Generated
0,3148,https://kuleuven-datathon-2023.s3.eu-central-1...,1
1,1480,https://kuleuven-datathon-2023.s3.eu-central-1...,1
2,182,https://kuleuven-datathon-2023.s3.eu-central-1...,1
3,2493,https://kuleuven-datathon-2023.s3.eu-central-1...,1
4,2497,https://kuleuven-datathon-2023.s3.eu-central-1...,1


In [283]:
Generated["AI_Generated"] = 1 
Generated2 = Generated.rename(columns = {'url':'image_url'})
Generated2.head()

,source_artwork,image_url,AI_Generated
0,3148,https://kuleuven-datathon-2023.s3.eu-central-1...,1
1,1480,https://kuleuven-datathon-2023.s3.eu-central-1...,1
2,182,https://kuleuven-datathon-2023.s3.eu-central-1...,1
3,2493,https://kuleuven-datathon-2023.s3.eu-central-1...,1
4,2497,https://kuleuven-datathon-2023.s3.eu-central-1...,1


# Movement

In [299]:
Movement.head()

,artist_id,movement_id
0,0.0,43.0
1,1.0,44.0
2,2.0,44.0
3,4.0,10.0
4,4.0,52.0


# Specialization 

In [168]:
Special.head()

,artist_id,specialty_id
0,0.0,0.0
1,0.0,1.0
2,0.0,2.0
3,0.0,3.0
4,1.0,2.0


# Artist

In [205]:
Artist.sample(5)

,id,name,url,summary,picture,birthplace,deathplace,birthdate,deathdate,cause_of_death
265,265,Frank Gresley,https://www.wikigallery.org//wiki/artist61226/...,Frank Gresley (1855–1936) was a British artist...,NaN,NaN,NaN,NaN,NaN,NaN
116,116,Auguste Joseph Marie De Mersseman,https://www.wikigallery.org//wiki/artist57946/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,52,William Merritt Chase,http://wikigallery.org/wiki/artist40281/Willia...,"William Merritt Chase (November 1, 1849 – Octo...",52.0,35.0,142.0,1849-11-01,1916-10-25,NaN
486,486,Myles Birket Foster,https://www.wikigallery.org//wiki/artist48895/...,Myles Birket Foster (4 February 1825 – 27 Mar...,NaN,NaN,NaN,NaN,NaN,NaN
326,326,French School,https://www.wikigallery.org//wiki/artist61277/...,Education in France is organized in a highly c...,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
Artist["cause_of_death"].isna().sum()/len(Artist) #cause of data column should be removed due to large number of NAs

0.9967532467532467

In [208]:
Artist["birthplace"].isna().sum()/len(Artist)

0.5

In [209]:
Artist["deathplace"].isna().sum()/len(Artist)

0.5275974025974026

In [210]:
Artist["birthdate"].isna().sum()/len(Artist)

0.6347402597402597

In [211]:
Artist["deathdate"].isna().sum()/len(Artist)

0.5616883116883117

In [284]:
Artist2 = Artist.iloc[:,[0,4,5,6,7,8]]
Artist2 = Artist2.rename(columns = {"id": "artist_id"})
Artist2['artist_id'] = Artist2['artist_id'].astype(float)
Artist2.head()

,artist_id,picture,birthplace,deathplace,birthdate,deathdate
0,0.0,0.0,0.0,342.0,1853-03-30,1890-07-29
1,1.0,1.0,1.0,343.0,1841-02-25,1919-12-03
2,2.0,2.0,2.0,344.0,1840-11-14,1926-12-05
3,3.0,3.0,3.0,NaN,1932-04-19,NaN
4,4.0,4.0,4.0,2.0,1824-05-11,1904-01-10


# Artist Education

In [223]:
Education.head()

,artist_id,academy_id
0,0.0,0.0
1,0.0,1.0
2,2.0,2.0
3,2.0,3.0
4,9.0,4.0


# Places

In [229]:
Places.head()

,id,name,parent
0,0,Zundert,213.0
1,1,Limoges,214.0
2,2,Paris,214.0
3,3,Medellín,215.0
4,4,Vesoul,216.0


In [298]:
Places2 = Places.iloc[:,[0,2]]
Places2 = Places2.rename(columns = {"id": "artist_id"})

# Text split

In [295]:
def FindNumber(inputstr):
    return inputstr.split("\\")[-1].split(".")[0]

In [ ]:
OutputReal["filename"] = OutputReal.apply(lambda row: FindNumber(row['filename']), axis = 1)

In [270]:
OutputReal2 = OutputReal.rename(columns = {"filename":"id", "logit": "real_logit", "time":"real_time"})

,id,real_logit,real_time
0,0,-8.378802,6.175854
1,1,-14.644643,6.019991
2,10,-6.296502,6.170134
3,100,-8.054534,6.067291
4,1000,-1.293363,5.589392


In [276]:
OutputReal2['id'] = OutputReal2['id'].astype(float) #change to float values to match main dataframe

In [296]:
OutputFake["filename"] = OutputFake.apply(lambda row: FindNumber(row['filename']), axis = 1)

In [297]:
OutputFake2 = OutputFake.rename(columns = {"filename":"id", "logit": "fake_logit", "time":"fake_time"})

In [277]:
OutputFake2['id'] = OutputFake2['id'].astype(float) #change to float values to match main dataframe

# Merge Dataframes

In [285]:
df_list1 = [Artwork2, Generated2]
df1 = pd.concat(df_list1)

In [286]:
df_list2 = [Movement, Special, df1, Artist2, Education, Places2]

df2 = reduce(lambda left, right: pd.merge(left, right, on = ["artist_id"], how = "outer"), df_list2)

In [235]:
df2.shape

(20788, 20)

In [287]:
df_list3 = [df2, OutputReal2, OutputFake2]

final_df = reduce(lambda left, right: pd.merge(left, right, on = ["id"], how = "outer"), df_list3)

In [293]:
final_df.shape

(20791, 20)

In [294]:
final_df.to_csv("merged_data_final.csv")